# Introduction to Linear Regression

-----

When you explore a data set, there are basic steps that should be followed. First, explore the data to be sure that missing values are properly handled, dates and times are encoded correctly, and that other strange artifacts are not present. Once the data has been cleaned, the next step is to compute descriptive statistics for the relevant features in the data, and to ensure they appear correct. Finally, joint distributions of different pairs of data should be visually explored to infer potential correlations. 

Once these steps are done, the next step is to model any potential correlations. In this notebook, we introduce one of the very basic modeling technique, **linear regression**, which constructs a simple model, such as
$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n$
from a data set. This model builds on assumptions, such as the features are linearly independent and any errors in the regression are normally distributed, to build a model from the independent variables (i.e., $x_1..x_n$) for the dependent variable ($y$). In some application areas, the independent variables are known as the predictors, while the dependent variable is known as the response variable. If only one feature is used ($x$), the technique is known as simple linear regression, while if more than one feature is used ($x_1, x_2, ..., x_n$), the technique is known as multiple linear regression.

Once a model has been built, it can be used for a variety of purposes, including prediction of new responses, understanding possible causation, or derivation of new physical laws and relationships.

In this lesson, we will use Python module `statsmodels` to construct a regression model on the mpg data set. We'd like to build a regression model to predict an automobile's fuel efficiency(mpg or mile per gallon) with multiple vehicle features.

## Table of Contents
[Data Preparation](#Data-Preparation)  

[Regression Model](#Regression-Model)

Before proceeding with an exploration of a demonstration data set, we first perform our standard module import.

-----

In [1]:
# Set up Notebook

%matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

# Use default white plot style
sns.set(style="white")

-----
[[Back to TOC]](#Table-of-Contents)

## Data Preparation

To demonstrate linear regression, we will use the mpg data set. This data set is included with the Seaborn module, making it easy to analyze. To begin, we load this data into our notebook. The first thing we do after loading the data set is to take a peek at the data set by displaying the first few rows. We will quickly get an idea of this data set. Then we use `info()` function to print the concise summary of the DataFrame.

-----
[waq]: https://en.wikipedia.org/wiki/Anscombe%27s_quartet

In [2]:
# Load data
mpg = pd.read_csv('mpg.csv')

# Display first few rows
mpg.head(5)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [3]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
mpg             398 non-null float64
cylinders       398 non-null int64
displacement    398 non-null float64
horsepower      392 non-null float64
weight          398 non-null int64
acceleration    398 non-null float64
model_year      398 non-null int64
origin          398 non-null object
name            398 non-null object
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


This data set has 9 columns. The column names are self-explanatory. There are 398 rows in the data set, and one column, horsepower, has 6 missing values. We will focus on linear regression in this lesson so we will simply drop the rows with missing values to get a clean data set. This is by no means the best approach since we will lose valuable data points. An alternative is estimating horsepower by other factors, i.e.. displacement and acceleration.

In [4]:
#simply drop missing values
mpg.dropna(inplace=True)
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 9 columns):
mpg             392 non-null float64
cylinders       392 non-null int64
displacement    392 non-null float64
horsepower      392 non-null float64
weight          392 non-null int64
acceleration    392 non-null float64
model_year      392 non-null int64
origin          392 non-null object
name            392 non-null object
dtypes: float64(4), int64(3), object(2)
memory usage: 30.6+ KB


-----
[[Back to TOC]](#Table-of-Contents)

### Descriptive Statistics

Our next step is to compute basic descriptive statistics for the features in this data set. To accomplish this, we can use the `describe` function on our Pandas DataFrame. 

-----

In [5]:
mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


-----
[[Back to TOC]](#Table-of-Contents)

## Regression Model

Now we've loaded the data set and cleaned it up. We also have some basic understandings of the data set. Since we want to build a model to predict fuel efficiency, the depend variable will be mpg. The next step is to pick dependent variables. All of the rest 8 columns can be independent variables. There are two types of variables in the data set, **categorical** and **continuous**.

- Categorical variables: the value is limited and usually based on a particular finite group. For example, origin.
- Continuous variables: numeric variables that have an infinite number of values between any two values, for example, horsepower.

In a regression model, a dependent variable must be continuous variable. Independent variables can be continuous or categorical. But when we pick categorical variables as independent variables, we normally need to create dummy variables for them. Take origin in this data set as an example, as shown below, origin has three unique values: usa, japan, and europe.

In [6]:
mpg.origin.unique()

array(['usa', 'japan', 'europe'], dtype=object)

To create dummy variables for origin, we can add two new columns to the data set, say origin_usa and origin_japan. 

- If origin = usa, orign_usa = 1 and origin_japan = 0;
- If origin = japan, origin_usa=0 and origin_japan = 1;
- If origin = europe, orgin_usa=0 and origin_japan = 0.

As shown above, even though there are three distinct values, we only need 2 dummy columns to cover all the cases.

The Python module `statsmodels` provides a way to create dummy variables for categorical variables, so we don't have to do it manually.

For this regression, mpg will be dependent variable, we will pick horsepower, weight and origin as independent variables. Among them, origin is categorical variable and the other two are continuous variables. The process to select independent variables can be very complicated and it's out of the scope of this lesson.

To construct the regression model, we first need to define a string-formula using column names(**column names can't have whitespaces**). For this data set and the variables we choose, the formula is defined as:

`mpg ~ horsepower + weight + C(origin)`

In this formula, mpg is dependent variable, horsepower, weight and origin are independent variable. `C(origin)` indicates that origin is categorical variable.

We will then construct the model with the string formula and the data set, then fit the model and display the summary of the regression result.

In [7]:
import statsmodels.formula.api as smf
formula = 'mpg ~ horsepower + weight + C(origin)'
model = smf.ols(formula, data=mpg)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     247.9
Date:                Tue, 11 Feb 2020   Prob (F-statistic):          2.44e-105
Time:                        11:13:02   Log-Likelihood:                -1112.2
No. Observations:                 392   AIC:                             2234.
Df Residuals:                     387   BIC:                             2254.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             43.7011      0.932     46.871      0.000      41.868      45.534
C(origin)[T.japan]     1.7811      0.696      2.558      0.011       0.412       3.150
C(origin)[T.usa]      -0.9611      0.640     -1.501      0.134      -2.220       0.298
horsepower            -0.0535      0.011     -4.875      0.000      -0.075      -0.032
weight                -0.0048      0.001     -8.878      0.000      -0.006      -0.004
==============================================================================
Omnibus:                       35.026   Durbin-Watson:                   0.914
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.308
Skew:                           0.658   Prob(JB):                     5.33e-11
Kurtosis:                       4.078   Cond. No.                     1.54e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.54e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Interpret Regression Result

First we have the dependent variable and the model and the method. OLS stands for [Ordinary Least Squares][ols] and the method “Least Squares” means that we’re trying to fit a regression line that would minimize the square of distance from the regression line.

The model has R-Squared value 0.719, which means 71.9% of the variance in our dependent variable(mpg) can be explained by this model.

Based on the coefficient values, we can construct the regression equation:

$mpg = 43.7 - 0.0535 horsepower - 0.0048 weight + 1.7811 origin.japan - 0.9611 origin.usa$

This reads:

1. Horsepower increase by 1, mpg will decrease by 0.0535
2. Weight increase by 1 pound, mgp will decrease by 0.0048
3. If origin is Japan, mpg will increase by 1.7811(comparing to European cars)
4. If origin is USA, mpg will decrease by 0.9611(comparing to European cars)

For an European car, origin.japan=0 and origin.usa=0, so the equation becomes:

$mpg = 43.7 - 0.0535 horsepower - 0.0048 weight$

Which means the impact of origin in incorporated in the intercept. That's why the impact of origin.japan and origin.usa is on top of European cars.

So far so good, we've got a model that can predict auto mpg with horsepower, weight and origin to certain extend($R^2=0.719$). But we need to look into the coefficient table further, especially t scores and p values($p>|t|$), for hypothesis test. $|t|>2$ or $P < 0.05$ indicates the coefficient is statistically significant(at 95% confidence). The coefficient of origin.usa has a p value 0.134, which means it's not statistically significant, or we can't reject the null-hypothesis that the coefficient is 0(at 95% confidence). This is also indicated by the 95% confidence interval of the coefficient, -2.220 to 0.298, which covers 0. Based on the p-value, we can say that Japanese cars are more fuel efficient than European cars holding horsepower and weight constant, but we can't say the same thing for American cars at 95% confidence level.

[ols]: http://setosa.io/ev/ordinary-least-squares-regression/


In [8]:
mpg.sample(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
156,16.0,8,400.0,170.0,4668,11.5,75,usa,pontiac catalina
97,18.0,6,225.0,105.0,3121,16.5,73,usa,plymouth valiant
131,32.0,4,71.0,65.0,1836,21.0,74,japan,toyota corolla 1200
302,34.5,4,105.0,70.0,2150,14.9,79,usa,plymouth horizon tc3
224,15.0,8,302.0,130.0,4295,14.9,77,usa,mercury cougar brougham
105,13.0,8,360.0,170.0,4654,13.0,73,usa,plymouth custom suburb
43,13.0,8,400.0,170.0,4746,12.0,71,usa,ford country squire (sw)
206,26.5,4,140.0,72.0,2565,13.6,76,usa,ford pinto
254,20.2,6,200.0,85.0,2965,15.8,78,usa,ford fairmont (auto)
372,27.0,4,151.0,90.0,2735,18.0,82,usa,pontiac phoenix


### Predict with the Regression Model

With the trained regression model, you can predict mpg for a car with its horsepower, weight and origin. You may predict with the formula generated from the model parameters:  
$mpg = 43.7 - 0.0535 horsepower - 0.0048 weight + 1.7811 origin.japan - 0.9611 origin.usa$  
For example, for a 3000 pound, 100 horse power American car, the mpg is predicted as 22.3 as shown below:  
$mpg = 43.7 - 0.0535 * 100 - 0.0048 * 3000 + 1.7811 * 0 - 0.9611 * 1 = 22.3$

You can also predict mpg for a group of cars with the model. To do this, we need to create a DataFrame to host the vehicle information. For example, we'd like to predict mpg for following cars:
- US car, 2500 pound, 100 horse power
- European car, 3500 pound, 150 horse power
- Japanese car, 2000 pound, 90 horse power

In the next two code cells, we first create a DataFrame for these cars, then predict mpg of these cars with the model. The result shows the mpg for the three cars. In the last code cell, we create a new mpg column in the DataFrame to display the result more clearly.

In [9]:
#Create DataFrame to host vechicle information
df_cars = pd.DataFrame({'origin':['usa', 'europe', 'japan'],
                        'weight':[2500, 3500, 2000],
                        'horsepower':[100, 150, 90]})
df_cars

,origin,weight,horsepower
0,usa,2500,100
1,europe,3500,150
2,japan,2000,90


In [10]:
#Predict mpg
result.predict(df_cars)

0    25.278677
1    18.719840
2    30.977730
dtype: float64

In [11]:
#Display result as DataFrame
df_cars['mpg'] = result.predict(df_cars)
df_cars

,origin,weight,horsepower,mpg
0,usa,2500,100,25.278677
1,europe,3500,150,18.719840
2,japan,2000,90,30.977730


----

### model_year, Categorical or Continuous?

Fuel efficiency improves over time. So it make sense to add model_year to the regression model as independent variable. model_year column contains numeric values, we can add it as a continuous variable. The forumla then looks like this:

`mpg ~ horsepower + weight + C(origin) + model_year`

However, this model assumes that fuel efficiency changes linearly over time, which is obviously not true. So it's better to add model_year into the regression model as a categorical feature, which leads to the following formula:

`mpg ~ horsepower + weight + C(origin) + C(model_year)`

We will compare these two models in the following code cells.

-----

In [12]:
formula = 'mpg ~ horsepower + weight + C(origin) + model_year'
model = smf.ols(formula, data=mpg)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     350.3
Date:                Tue, 11 Feb 2020   Prob (F-statistic):          5.21e-141
Time:                        11:13:02   Log-Likelihood:                -1025.7
No. Observations:                 392   AIC:                             2063.
Df Residuals:                     386   BIC:                             2087.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            -15.2661      4.099     -3.724      0.000     -23.326      -7.206
C(origin)[T.japan]     0.3279      0.568      0.577      0.564      -0.789       1.444
C(origin)[T.usa]      -1.9546      0.519     -3.769      0.000      -2.974      -0.935
horsepower            -0.0085      0.009     -0.908      0.365      -0.027       0.010
weight                -0.0056      0.000    -12.622      0.000      -0.006      -0.005
model_year             0.7544      0.052     14.631      0.000       0.653       0.856
==============================================================================
Omnibus:                       32.098   Durbin-Watson:                   1.250
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.453
Skew:                           0.519   Prob(JB):                     9.09e-13
Kurtosis:                       4.523   Cond. No.                     7.54e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.54e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

We can see that adding model_year to the model increase $R^2$ from 0.719 to 0.819. The coefficient of model_year is 0.7544, which means mpg increases by 0.7544 each year. The coefficient is also statistically significant. Now let's see what we get when we add model_year to the regression model as a categorical variable.

In [13]:
formula = 'mpg ~ horsepower + weight + C(origin) + C(model_year)'
model = smf.ols(formula, data=mpg)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     134.8
Date:                Tue, 11 Feb 2020   Prob (F-statistic):          1.89e-144
Time:                        11:13:02   Log-Likelihood:                -986.88
No. Observations:                 392   AIC:                             2008.
Df Residuals:                     375   BIC:                             2075.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              38.6291      0.944     40.903      0.000      36.772      40.486
C(origin)[T.japan]      0.1220      0.528      0.231      0.817      -0.915       1.159
C(origin)[T.usa]       -1.9380      0.486     -3.988      0.000      -2.894      -0.982
C(model_year)[T.71]     0.9104      0.867      1.050      0.294      -0.794       2.615
C(model_year)[T.72]    -0.3460      0.844     -0.410      0.682      -2.005       1.313
C(model_year)[T.73]    -0.7063      0.771     -0.916      0.360      -2.222       0.809
C(model_year)[T.74]     1.2579      0.902      1.394      0.164      -0.516       3.032
C(model_year)[T.75]     0.6394      0.885      0.723      0.470      -1.100       2.379
C(model_year)[T.76]     1.4144      0.849      1.666      0.096      -0.255       3.083
C(model_year)[T.77]     2.8481      0.865      3.294      0.001       1.148       4.548
C(model_year)[T.78]     2.6981      0.824      3.276      0.001       1.078       4.318
C(model_year)[T.79]     5.1060      0.874      5.842      0.000       3.387       6.825
C(model_year)[T.80]     9.1671      0.922      9.938      0.000       7.353      10.981
C(model_year)[T.81]     6.5309      0.910      7.178      0.000       4.742       8.320
C(model_year)[T.82]     8.2093      0.884      9.286      0.000       6.471       9.948
horsepower             -0.0162      0.009     -1.710      0.088      -0.035       0.002
weight                 -0.0051      0.000    -11.594      0.000      -0.006      -0.004
==============================================================================
Omnibus:                       22.222   Durbin-Watson:                   1.501
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.051
Skew:                           0.335   Prob(JB):                     7.39e-10
Kurtosis:                       4.458   Cond. No.                     4.85e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.85e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The new model has $R^2=0.852$. And if we look into the coefficients of dummy variables of model_year, we can see that from 1971 to 1976, there's not much improvement in vehicle mpg; from 1977 to 1982, vehicle mpg has significant improvements every year. This model gives better $R^2$ and reveals more insight in vehicle mpg changes over time. One drawback of this model is that we can't predict vehicle mpg with it if the vehicle is made after 1982. But overall, we can see that adding model_year as categorical variable gives us a better regression model.

Notice that when we add model_year to the regression model, the coefficient of horsepower is no longer significant. This indicates that there could be a [multicollinearity][mcl] problem, or horsepower could somewhat be explained by other independent variables. This is a very important concept in regression but it's out of the scope of this lesson.

---
[mcl]:https://en.wikipedia.org/wiki/Multicollinearity

-----

<font color='red' size = '5'> Student Exercise </font>

In the empty **Code** cell, drop horsepower from the regression model, only include weight, origin and model_year. What's the impact of this change?



-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. Wikipedia has an excellent discussion on [simple linear regression][1].
2. Wikipedia discussion on [multicollinearity][2]

-----

[1]: https://en.wikipedia.org/wiki/Simple_linear_regression
[2]:https://en.wikipedia.org/wiki/Multicollinearity

**&copy; 2019: Gies College of Business at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode 